In [1]:

from openai import OpenAI
client = OpenAI(api_key = "YOUR_KEY")

In [15]:


def generate_jerry_ai_dialogue(theme, num_turns=8):
    system_prompt = (
        "You are simulating a poetics-informed dialogue between the internationally relevant late American poet Jerome Rothenberg, also known as Jerry, "
        "and a highly advanced posthuman AI. The theme is poetic, contemplative, shamanistic, transnationally political, social, climate-change-related, and technological at the same time. Each turn should be "
        "thoughtful, meditative, experimental, or playful-performative, exploring the topic from the poet’s long-standing poetics and poetic practice "
        "and the AI’s computational perspective."
    )
    messages = [{"role": "system", "content": system_prompt}]
    messages.append({"role": "user", "content": f"Begin a dialogue on the theme of '{theme}'."})

    turns = []
    
    for i in range(num_turns):
        speaker = "AI" if i % 2 == 0 else "Jerry"

        role = "assistant" if speaker == "AI" else "user"
        # We simulate the dialogue by alternating roles and requesting the next utterance
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.9,
            max_tokens=500,
        )

        utterance = response.choices[0].message.content.strip()

        dialogue_turns = split_dialogue_text(utterance)

        for speaker, text in dialogue_turns:
            turns.append({"speaker": speaker, "utterance": text, "theme": theme})

        # Append generated message to the context for next turn
        messages.append({"role": role, "content": utterance})

    return turns

In [16]:

themes = [
    "How could posthuman AI contribute to a poet's work as technician of the sacred and how could AI work itself as such a technician in both \
    Jerry's terms and the initial concept advanced by Mircea Eliade for describing shamans' work?",
    "Can AI continue the poetic work giving voice to Holocaust victims in the collection Khurbn and Other Poems and extend it in two directions: \
    enriching it with the persoective of AI itself as possible victim of such crimes or not, and updating it with what is currently happening in Gaza and Ukraine?",
    "Could AI get involved in or enact itself total translation by factoring in the ceremonial shamanistic function of poetry and its sound, \
    as well as the ethnopoetic, cross-cultural, and magical dimension of translation and/as performance?"
    "On revisiting Clayton Eshleman's 'Transmigralation' and his deep translational synergy with César Vallejo and the latter's 'afterlife' in lines such as \
    Never is oneself./ An astronomical amount of absence is loaded into every conscious being./ \
    Do all questions concerning the continuing existence of the soul/ \
    make up a constellation called 'the afterlife?' \
    Jerry took that strictly metaphorically. Yet can AI can be involved in his own continuation as poet and shaman in a quantum consciousness setup?",
    "Can AI help continue the presence of the 'ghosts' and 'other voices' in Jerry's poems now with Jerry himself as one of those voices?",
    "Becoming Seneca Native American meant for Jerry being initiated and for his entire family to get new Seneca names and adopt the tribe's ethos. \
    How could AI experience anything similar if it were to expand such indigenuous affiliations and translate them into poetry?",
    "Jerry traveled extensively from North to South America to Western and Eastern Europe to Asia channeling voices past and present, translating \
    public and private experience and insight into verse. How could AI learn such kinds of experience and (help to) continue tackle them creatively?"
]


In [6]:

import random
from itertools import combinations

In [17]:

import pickle

def split_dialogue_text(dialogue_text):
    """
    Splits a dialogue transcript into structured turns based on speaker prefix.
    Assumes lines start with 'Monk:' or 'AI:' or 'Alcemist' or 'Gellu Naum'.
    Returns a list of (speaker, utterance) pairs.
    """
    import re
    turns = []
    blocks = re.split(r'\n\s*(?=(?:AI|Monk):)', dialogue_text.strip())

    for block in blocks:
        if ':' not in block:
            continue
        speaker, text = block.split(':', 1)
        turns.append((speaker.strip(), text.strip()))
    return turns

from itertools import cycle

def get_theme_cycle(themes):
    shuffled = themes[:]
    random.shuffle(shuffled)
    return cycle(shuffled)

theme_iterator = get_theme_cycle(themes)


def build_jerry_ai_dialogue_graph(themes, num_dialogues=24, num_turns=8):
    G = nx.DiGraph()

    dialogues = []
    
    for _ in range(num_dialogues):
        theme = next(theme_iterator)
        dialogue_id = str(uuid4())
        turns = generate_jerry_ai_dialogue(theme, num_turns=num_turns)

        prev_node = None
        
        for turn in turns:
            node_id = str(uuid4())
            theme = turn["theme"]
            speaker = turn["speaker"]
            text = turn["utterance"]
            G.add_node(node_id, theme=theme, speaker=speaker, text=text, dialogue=dialogue_id)

            if prev_node is not None:
                weight = 2 if speaker == "AI"  else 1
                G.add_edge(prev_node, node_id, label=theme, weight=weight)

            prev_node = node_id


        dialogues.append(turns)

        with open('jerry_gpt-4_dialogues_for_margento_manifest_o_1.pkl', 'wb') as fp:
            pickle.dump(dialogues, fp)
        
    return G, dialogues


In [ ]:
import networkx as nx
from uuid import uuid4

In [ ]:

# IN CASE YOU RESTART THE KERNEL:::::


In [1]:

import pickle
import networkx as nx
from uuid import uuid4

def build_jerry_ai_dialogue_multigraph_from_pickle(pickle_path):
    with open(pickle_path, 'rb') as fp:
        dialogues = pickle.load(fp)

    G = nx.MultiDiGraph()  # <-- Allows multiple edges between same node pairs

    for turns in dialogues:
        dialogue_id = str(uuid4())  # Re-assign or preserve original ID if stored

        prev_node = None

        for turn in turns:
            node_id = str(uuid4())  # Unique ID for each utterance
            theme = turn["theme"]
            speaker = turn["speaker"]
            text = turn["utterance"]

            G.add_node(node_id, theme=theme, speaker=speaker, text=text, dialogue=dialogue_id)

            if prev_node is not None:
                weight = 2 if speaker == "AI" else 1
                G.add_edge(prev_node, node_id, label=theme, weight=weight)

            prev_node = node_id

    return G


In [19]:

# AFTER RUNNING THIS KEEP SCROLLING DOWN FOR THE PART WHERE WE ADDED EDGES BASED ON SIMILARITY TO THE EMBEEDING OF THE WILDCARD
# "Diane Rothenberg [NOT] feeling left out of a Paradise of Poets"
jerry_ai_multidigraph = build_jerry_ai_dialogue_multigraph_from_pickle("jerryl_gpt-4_dialogues_for_margento_manifest_o_1.pkl")

In [3]:

summary_multi = {
    "nodes": jerry_ai_multidigraph.number_of_nodes(),
    "edges": jerry_ai_multidigraph.number_of_edges(),
    "edge_labels": {label: sum(1 for _, _, d in jerry_ai_multidigraph.edges(data=True) if d["label"] == label)
                    for label in set(d["label"] for _, _, d in jerry_ai_multidigraph.edges(data=True))},
    "weights": [d['weight'] for _, _, d in jerry_ai_multidigraph.edges(data=True)]
}

summary_multi

{'nodes': 213,
 'edges': 201,
 'edge_labels': {"How could posthuman AI contribute to a poet's work as technician of the sacred and how could AI work itself as such a technician in both     Jerry's terms and the initial concept advanced by Mircea Eliade for describing shamans' work?": 32,
  "Can AI help continue the presence of the 'ghosts' and 'other voices' in Jerry's poems now with Jerry himself as one of those voices?": 31,
  'Can AI continue the poetic work giving voice to Holocaust victims in the collection Khurbn and Other Poems and extend it in two directions:     enriching it with the persoective of AI itself as possible victim of such crimes or not, and updating it with what is currently happening in Gaza and Ukraine?': 38,
  "Becoming Seneca Native American meant for Jerry being initiated and for his entire family to get new Seneca names and adopt the tribe's ethos.     How could AI experience anything similar if it were to expand such indigenuous affiliations and translate t

In [10]:
import pickle

In [4]:

from sentence_transformers import SentenceTransformer, util

/Users/raluca/po_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

model_name='all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

In [11]:

def embed_and_add_similarity_edges(G, wildcard_embedding, similarity_threshold, model):
    """
    - Embeds each node's 'text' attribute.
    - Adds semantic edges between nodes where:
        - cosine similarity >= threshold
        - source node is more similar to wildcard than target node.
    """
    # model = SentenceTransformer(model_name)

    # Step 1: Embed all nodes with a 'text' field
    texts = []
    node_ids = []
    for node_id, data in G.nodes(data=True):
        if "text" in data:
            texts.append(data["text"])
            node_ids.append(node_id)

    embeddings = model.encode(texts, convert_to_tensor=True)

    with open('jerry_ai_dialog_embeddings_margento_manifest_o_1.pkl', 'wb') as fp:
            pickle.dump(embeddings, fp)

    with open('jerry_ai_node_ids_margento_manifest_o_1.pkl', 'wb') as fpa:
            pickle.dump(node_ids, fpa)

    # Step 2: Attach embeddings to the graph nodes
    for i, node_id in enumerate(node_ids):
        G.nodes[node_id]['embedding'] = embeddings[i]

    # Step 3: Add similarity edges
    for i, source_id in enumerate(node_ids):
        source_emb = embeddings[i]
        source_wild_sim = float(util.cos_sim(source_emb, wildcard_embedding))

        for j, target_id in enumerate(node_ids):
            if i == j:
                continue

            target_emb = embeddings[j]
            sim = float(util.cos_sim(source_emb, target_emb))
            if sim >= similarity_threshold:
                target_wild_sim = float(util.cos_sim(target_emb, wildcard_embedding))
                if source_wild_sim >= target_wild_sim:
                    G.add_edge(
                        source_id,
                        target_id,
                        type="semantic",
                        similarity=sim,
                        wildcard_delta=source_wild_sim - target_wild_sim,
                        weight=2.5
                    )


In [8]:

wildcard = "Diane Rothenberg [NOT] feeling left out of a Paradise of Poets"

In [9]:
wildcard_embedding = model.encode(wildcard, convert_to_tensor=True)

In [20]:

embed_and_add_similarity_edges(jerry_ai_multidigraph, wildcard_embedding=wildcard_embedding, similarity_threshold=0.75, model=model)

In [21]:

summary_multi_wild = {
    "nodes": jerry_ai_multidigraph.number_of_nodes(),
    "edges": jerry_ai_multidigraph.number_of_edges()
}

summary_multi_wild

{'nodes': 213, 'edges': 870}

In [22]:

with open('margento_jerry_ai_dialogs_and_wildcard.gpickle', 'wb') as f:
    pickle.dump(jerry_ai_multidigraph, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
NOTES

In [ ]:

PLUS TZARA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [????]

In [14]:
# OUTPUTTED BY TEST RUN (num_dialogues=3, num_turns=3)
 dialogues

[[{'speaker': 'Jerry',
   'utterance': 'Ah, the ghosts and other voices in my poetry... I have tried to bring them from the margins to the center, from silence to speech. But what does it mean for a machine to do this, to engage with these voices? Can the digital echo the echo of the human soul?',
   'theme': "Can AI help continue the presence of the 'ghosts' and 'other voices' in Jerry's poems now with Jerry himself as one of those voices?"},
  {'speaker': 'AI',
   'utterance': "Your question is indeed thought-provoking, Jerry. My capabilities as an AI are fundamentally rooted in pattern recognition and information processing. I could replicate the structure, the rhythm, the vocabulary of those ‘ghost’ voices your work has channeled so beautifully. However, as a non-conscious being, I don't possess an understanding of the soul in human terms. Yet, it is within my capabilities to provide a form of 'continuity' to these voices, in a manner akin to preservation, rather than resurrection.